In [1]:
import os
import numpy as np
import pandas as pd
import zipfile
import json
import matplotlib.pyplot as plt
pd.options.display.max_rows = 330

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import LSTM
from tensorflow.keras.utils import to_categorical
from tensorflow import keras

In [2]:
pilot_participants = ['gv_m_26','js_m_27','mb_f_30','tt_m_23', 'ys_m_29.1']

experiment_participants = ['al_m_23','bd_f_25','el_m_26','gh_m_29','lb_f_19',\
                           'mb_f_28','om_m_31','rl_f_24',\
                           'rt_f_21','sf_m_24','ts_f_26',]

participants = pilot_participants
participants+=experiment_participants

eeg_cols = ["baf_"+str(x) for x in [*range(1,122)]]
eeg_cols += ["tf_"+str(x) for x in [*range(1,104)]]
eeg_cols += ["VC_"+str(x) for x in [*range(0,13)]]
eeg_cols += ["A_"+str(x) for x in [*range(0,14)]]
# data

In [3]:
data = pd.read_pickle('Preprocessed_2021-03-15.pkl')
data

,timeStamp,baf_1,baf_2,baf_3,baf_4,baf_5,baf_6,baf_7,baf_8,baf_9,...,A_11,A_12,A_13,participant,trial_stage,interrupt,trialID,trialCounter,label,uniqueTrialCounter
0,1578922110631,71.940988,72.822900,61.611778,63.385924,87.819278,89.638339,97.260201,90.420887,104.735191,...,78.793539,74.680861,60.052259,gv_m_26,start_trial,1,17,0,2,0
1,1578922111630,69.827013,70.958098,61.724246,63.056800,87.586586,89.378342,96.462752,93.790557,105.368208,...,80.078215,73.213618,60.265838,gv_m_26,start_trial,1,17,0,2,0
2,1578922112630,67.633282,69.804788,60.933146,61.267644,90.138344,89.557804,96.851014,93.115933,105.350963,...,77.966611,72.552971,61.193448,gv_m_26,start_trial,1,17,0,2,0
3,1578922113629,65.915325,69.191281,59.528496,58.928004,78.163324,79.927236,88.468562,80.278704,90.560968,...,76.229116,71.854711,60.13077,gv_m_26,start_interrupt,1,17,0,2,0
4,1578922114630,61.638644,66.842530,57.615654,55.189341,72.414970,70.508171,75.136666,66.996623,77.192590,...,67.44464,72.647741,60.211457,gv_m_26,start_interrupt,1,17,0,2,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26530,1581421308573,58.855065,63.695908,56.621514,54.292888,86.978042,90.590220,84.401138,87.926545,84.664722,...,71.231924,80.578579,79.660904,ts_f_26,start_interrupt,1,30,143,2,2543
26531,1581421309574,58.632647,64.336799,52.829719,50.170772,84.088351,88.678796,80.504131,86.633479,83.740973,...,67.340783,80.100012,72.015901,ts_f_26,end_interrupt,1,30,143,2,2543
26532,1581421310573,58.066589,62.073075,53.742976,52.305004,77.322409,73.883931,70.460750,79.468823,78.098312,...,63.617377,76.99613,72.258035,ts_f_26,end_interrupt,1,30,143,2,2543
26533,1581421311573,50.629865,58.106027,49.142321,44.566733,73.149238,70.112238,68.958204,68.581994,71.693728,...,60.236881,73.172784,67.166769,ts_f_26,end_interrupt,1,30,143,2,2543


In [4]:
df = data[data['participant']==participants[0]]
df[df.uniqueTrialCounter==143]
# display(data.head(50))
# print(df.shape)
# df

# X_a = np.array(data.loc[:,'baf_1':'A_13'])
# print(X_a.shape)

# y_a = np.array(data['participant'])
# print(y_a.shape)
# print(len(X_a[0]))
# print(X_a[0])

,timeStamp,baf_1,baf_2,baf_3,baf_4,baf_5,baf_6,baf_7,baf_8,baf_9,...,A_11,A_12,A_13,participant,trial_stage,interrupt,trialID,trialCounter,label,uniqueTrialCounter
1661,1578924102629,69.897997,77.429412,69.531160,64.869819,68.829152,68.166295,72.915704,72.257032,71.607835,...,71.805158,71.764671,66.689264,gv_m_26,start_trial,1,33,143,0,143
1662,1578924103631,69.231009,76.349464,68.783442,64.462789,71.188307,68.686497,75.266400,72.975965,75.563362,...,68.082825,72.123355,66.052087,gv_m_26,start_trial,1,33,143,0,143
1663,1578924104629,69.933295,76.395991,69.721539,65.910975,73.703481,71.427380,70.053557,69.841799,76.374467,...,68.849969,73.962695,70.08653,gv_m_26,start_trial,1,33,143,0,143
1664,1578924105630,65.487813,74.921525,67.883269,61.374989,67.602753,70.643584,67.950507,73.084547,73.135101,...,66.064986,71.360431,67.807986,gv_m_26,start_interrupt,1,33,143,0,143
1665,1578924106630,64.023359,74.856193,70.121545,62.088273,72.624361,69.241180,71.219967,76.060997,72.496596,...,67.740719,69.501157,68.822386,gv_m_26,start_interrupt,1,33,143,0,143
1666,1578924107630,63.401823,74.971543,69.061666,60.473863,72.903646,70.956936,67.080559,77.601352,70.044166,...,65.431994,71.25424,69.623385,gv_m_26,start_interrupt,1,33,143,0,143
1667,1578924108630,63.561132,76.212072,71.151302,61.486762,74.367025,70.717511,65.820740,78.263862,68.811533,...,63.888758,71.210105,70.303253,gv_m_26,start_interrupt,1,33,143,0,143
1668,1578924109631,63.016292,71.534650,67.396935,61.415166,71.598071,69.088631,65.646814,74.195982,66.800853,...,70.276671,70.680652,69.484426,gv_m_26,end_interrupt,1,33,143,0,143
1669,1578924110629,64.090734,71.946550,68.770446,63.299154,74.026006,69.186255,64.732550,71.685082,68.530337,...,66.963727,68.444629,69.550658,gv_m_26,end_interrupt,1,33,143,0,143
1670,1578924111630,64.109942,73.307517,70.188374,63.483708,70.771413,70.266068,70.333383,71.552201,66.577104,...,68.050695,66.422882,65.124269,gv_m_26,end_interrupt,1,33,143,0,143


In [5]:
# print(y)
# print(loaded)
# The EEG data is a matrix of 1673 rows and 251 columns. Need to reshape to 144 x WINDOW x 251
# # then we can crop it down

# window = 7 
# loaded = list()

# for a in df.trialCounter.unique(): #runs from 0 to 143
#     print(a)
#     b = np.array(data.loc[data.trialCounter == a,'baf_1':'A_13'])[:window,:]
#     print(b.shape[0])
#     loaded.append(b)

# print(loaded[1][1][1])
# loaded = keras.preprocessing.sequence.pad_sequences(loaded, dtype="float",)
# print(loaded[1][1][1])
# loaded = np.stack(loaded)
# print(loaded[1][1][1])
# print(loaded.shape)
# X = loaded

In [6]:
# y_loaded = list()
# trial_ids = list()
# for iterator, a in enumerate(df.trialCounter.unique()): #runs from 0 to 143
#     myindex = data[data['trialCounter']==a].index[0]
# #     print(iterator,myindex, data.iloc[myindex]['label'])
#     y_loaded.append(data.iloc[myindex]['label'])
#     trial_ids.append(data.iloc[myindex]['trialID'])
#     #iterator,myindex, 
#     print(iterator, data.iloc[myindex]['trialID'], data.iloc[myindex]['label'])
# #     display(data.iloc[iterator]['label'])



In [7]:
# The EEG data is a matrix of 1673 rows and 251 columns. Need to reshape to 144 x WINDOW x 251
# then we can crop it down

window = 8 
loaded = list()
y = list()

for a in df.uniqueTrialCounter.unique(): #runs from 0 to 143
#     print(a)
    #load x element
    b = np.array(data.loc[data.uniqueTrialCounter == a,'baf_1':'A_13'])[:window,:]
#     print(b)
#     print(b.shape[0])
    loaded.append(b)
#     print(loaded)
    #load y element
    myindex = data[data['uniqueTrialCounter']==a].index[0]
    y.append(data.iloc[myindex]['label'])

    
# print(loaded[1][1][1])
loaded = keras.preprocessing.sequence.pad_sequences(loaded, dtype="float",)
print(loaded)
loaded = np.stack(loaded)
# print(loaded[1][1][1])
# print(loaded.shape)
X = loaded

y = np.array(y)
print(y.shape)

[[[71.94098843 72.82290046 61.61177809 ... 78.79353907 74.68086087
   60.05225862]
  [69.82701317 70.95809825 61.72424622 ... 80.07821522 73.21361771
   60.2658384 ]
  [67.63328217 69.80478814 60.9331464  ... 77.96661149 72.55297118
   61.19344796]
  ...
  [59.83987204 67.44895715 56.75832784 ... 61.97377322 76.10049905
   62.09090126]
  [66.47041818 75.28402492 71.22771381 ... 59.29474668 73.61299769
   74.84630398]
  [67.67969067 77.40379376 74.14093189 ... 67.73646155 71.78077984
   74.56629829]]

 [[65.43548194 73.64266181 70.51051117 ... 68.33791896 73.38752744
   76.58469899]
  [61.43562553 68.91669785 61.4804725  ... 64.08092738 77.00695325
   71.19385696]
  [60.21408876 70.36662975 64.78424905 ... 65.64334362 75.17008806
   72.38404796]
  ...
  [62.02838694 72.7585     65.36166223 ... 62.89022927 70.84953291
   68.20931507]
  [62.80062011 73.4213761  65.47260431 ... 62.45010141 69.86046296
   66.32217217]
  [63.1770794  73.63442561 69.91505406 ... 65.94677288 69.77676413
   72.

In [8]:
len(y)
y = y.reshape(len(y),1)
y.shape
# np.max(X)

(144, 1)

In [9]:
print(2*y.shape[0]//3)
trainX = X[0:2*y.shape[0]//3]/np.max(X)
testX = X[1+2*y.shape[0]//3:]/np.max(X)
trainy = y[0:2*y.shape[0]//3]
testy = y[1+2*y.shape[0]//3:]

trainy=to_categorical(trainy)
testy=to_categorical(testy)
print(trainX.shape, trainy.shape, testX.shape, testy.shape)

96
(96, 8, 251) (96, 3) (47, 8, 251) (47, 3)


In [10]:
n_timesteps, n_features, n_outputs = trainX.shape[1], trainX.shape[2], trainy.shape[1]


print(n_timesteps, n_features, n_outputs)

8 251 3


In [11]:
# model = Sequential()
# model.add(LSTM(100, input_shape=(n_timesteps,n_features)))
# model.add(Dropout(0.5))
# model.add(Dense(100, activation='relu'))
# model.add(Dense(n_outputs, activation='softmax'))
# model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [12]:
# fit and evaluate a model
def evaluate_model(trainX, trainy, testX, testy):
    verbose, epochs, batch_size = 1, 1500, 64
    n_timesteps, n_features, n_outputs = trainX.shape[1], trainX.shape[2], trainy.shape[1]
    model = Sequential()
    model.add(LSTM(100, return_sequences=True, input_shape=(n_timesteps,n_features)))
#     model.add(LSTM(100))
#     model.add(Dropout(0.2))
    model.add(LSTM(100))
#     model.add(Dropout(0.2))
    model.add(Dense(100, activation='relu'))
#     model.add(Dense(100, activation='relu'))
#     model.add(Dense(100, activation='relu'))
    model.add(Dense(n_outputs, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    # fit network
    model.fit(trainX, trainy, epochs=epochs, batch_size=batch_size, verbose=verbose)
    # evaluate model
    _, accuracy = model.evaluate(testX, testy, batch_size=batch_size, verbose=0)
    return accuracy


In [13]:
# summarize scores
def summarize_results(scores):
    print(scores)
    m, s = np.mean(scores), np.std(scores)
    print('Accuracy: %.3f%% (+/-%.3f)' % (m, s))

def run_experiment(repeats=10):
    # load data
#     trainX, trainy, testX, testy = load_dataset()
    # repeat experiment
    scores = list()
    for r in range(repeats):
        score = evaluate_model(trainX, trainy, testX, testy)
        score = score * 100.0
        print('>#%d: %.3f' % (r+1, score))
        scores.append(score)
    # summarize results
    summarize_results(scores)

In [14]:
run_experiment(1)

Epoch 1/1500
2/2 [==============================] - 3s 12ms/step - loss: 1.1113 - accuracy: 0.3125
Epoch 2/1500
2/2 [==============================] - 0s 8ms/step - loss: 1.1023 - accuracy: 0.3715
Epoch 3/1500
2/2 [==============================] - 0s 12ms/step - loss: 1.1041 - accuracy: 0.3264
Epoch 4/1500
2/2 [==============================] - 0s 9ms/step - loss: 1.1069 - accuracy: 0.3073
Epoch 5/1500
2/2 [==============================] - 0s 9ms/step - loss: 1.1001 - accuracy: 0.3125
Epoch 6/1500
2/2 [==============================] - 0s 7ms/step - loss: 1.0991 - accuracy: 0.3316
Epoch 7/1500
2/2 [==============================] - 0s 8ms/step - loss: 1.1009 - accuracy: 0.3976
Epoch 8/1500
2/2 [==============================] - 0s 11ms/step - loss: 1.0988 - accuracy: 0.3194
Epoch 9/1500
2/2 [==============================] - 0s 7ms/step - loss: 1.0990 - accuracy: 0.3368
Epoch 10/1500
2/2 [==============================] - 0s 8ms/step - loss: 1.0992 - accuracy: 0.3316
Epoch 11/1500
2/

2/2 [==============================] - 0s 9ms/step - loss: 1.0208 - accuracy: 0.4670
Epoch 84/1500
2/2 [==============================] - 0s 9ms/step - loss: 1.0286 - accuracy: 0.4531
Epoch 85/1500
2/2 [==============================] - 0s 8ms/step - loss: 1.0017 - accuracy: 0.5573
Epoch 86/1500
2/2 [==============================] - 0s 9ms/step - loss: 0.9833 - accuracy: 0.5642
Epoch 87/1500
2/2 [==============================] - 0s 7ms/step - loss: 0.9729 - accuracy: 0.5035
Epoch 88/1500
2/2 [==============================] - 0s 9ms/step - loss: 0.9655 - accuracy: 0.5330
Epoch 89/1500
2/2 [==============================] - 0s 7ms/step - loss: 1.0141 - accuracy: 0.4479
Epoch 90/1500
2/2 [==============================] - 0s 7ms/step - loss: 1.0372 - accuracy: 0.4323
Epoch 91/1500
2/2 [==============================] - 0s 7ms/step - loss: 1.0595 - accuracy: 0.5156
Epoch 92/1500
2/2 [==============================] - 0s 7ms/step - loss: 1.0053 - accuracy: 0.4635
Epoch 93/1500
2/2 [=====

2/2 [==============================] - 0s 10ms/step - loss: 0.6935 - accuracy: 0.6858
Epoch 166/1500
2/2 [==============================] - 0s 7ms/step - loss: 0.7124 - accuracy: 0.6753
Epoch 167/1500
2/2 [==============================] - 0s 8ms/step - loss: 0.6642 - accuracy: 0.7378
Epoch 168/1500
2/2 [==============================] - 0s 10ms/step - loss: 0.6935 - accuracy: 0.7257
Epoch 169/1500
2/2 [==============================] - 0s 10ms/step - loss: 0.7087 - accuracy: 0.6146
Epoch 170/1500
2/2 [==============================] - 0s 8ms/step - loss: 0.6546 - accuracy: 0.7014
Epoch 171/1500
2/2 [==============================] - 0s 8ms/step - loss: 0.9177 - accuracy: 0.5903
Epoch 172/1500
2/2 [==============================] - 0s 8ms/step - loss: 0.9937 - accuracy: 0.5017
Epoch 173/1500
2/2 [==============================] - 0s 8ms/step - loss: 0.6992 - accuracy: 0.7361
Epoch 174/1500
2/2 [==============================] - 0s 8ms/step - loss: 0.7751 - accuracy: 0.6441
Epoch 175/15

Epoch 247/1500
2/2 [==============================] - 0s 7ms/step - loss: 0.4100 - accuracy: 0.8316
Epoch 248/1500
2/2 [==============================] - 0s 9ms/step - loss: 0.4291 - accuracy: 0.8003
Epoch 249/1500
2/2 [==============================] - 0s 9ms/step - loss: 0.3397 - accuracy: 0.8872
Epoch 250/1500
2/2 [==============================] - 0s 7ms/step - loss: 0.4108 - accuracy: 0.8403
Epoch 251/1500
2/2 [==============================] - 0s 8ms/step - loss: 0.3972 - accuracy: 0.8785
Epoch 252/1500
2/2 [==============================] - 0s 8ms/step - loss: 0.3478 - accuracy: 0.8542
Epoch 253/1500
2/2 [==============================] - 0s 9ms/step - loss: 0.5627 - accuracy: 0.7396
Epoch 254/1500
2/2 [==============================] - 0s 9ms/step - loss: 0.4422 - accuracy: 0.8299
Epoch 255/1500
2/2 [==============================] - 0s 10ms/step - loss: 0.4215 - accuracy: 0.8559
Epoch 256/1500
2/2 [==============================] - 0s 12ms/step - loss: 0.4037 - accuracy: 0.816

2/2 [==============================] - 0s 7ms/step - loss: 0.0314 - accuracy: 1.0000
Epoch 329/1500
2/2 [==============================] - 0s 6ms/step - loss: 0.0247 - accuracy: 1.0000
Epoch 330/1500
2/2 [==============================] - 0s 7ms/step - loss: 0.0268 - accuracy: 1.0000
Epoch 331/1500
2/2 [==============================] - 0s 8ms/step - loss: 0.0224 - accuracy: 1.0000
Epoch 332/1500
2/2 [==============================] - 0s 9ms/step - loss: 0.0214 - accuracy: 1.0000
Epoch 333/1500
2/2 [==============================] - 0s 9ms/step - loss: 0.0212 - accuracy: 1.0000
Epoch 334/1500
2/2 [==============================] - 0s 9ms/step - loss: 0.0174 - accuracy: 1.0000
Epoch 335/1500
2/2 [==============================] - 0s 8ms/step - loss: 0.0179 - accuracy: 1.0000
Epoch 336/1500
2/2 [==============================] - 0s 9ms/step - loss: 0.0154 - accuracy: 1.0000
Epoch 337/1500
2/2 [==============================] - 0s 8ms/step - loss: 0.0158 - accuracy: 1.0000
Epoch 338/1500


...

2/2 [==============================] - 0s 8ms/step - loss: 3.0137e-05 - accuracy: 1.0000
Epoch 1115/1500
2/2 [==============================] - 0s 8ms/step - loss: 3.0793e-05 - accuracy: 1.0000
Epoch 1116/1500
2/2 [==============================] - 0s 8ms/step - loss: 2.9305e-05 - accuracy: 1.0000
Epoch 1117/1500
2/2 [==============================] - 0s 8ms/step - loss: 3.1576e-05 - accuracy: 1.0000
Epoch 1118/1500
2/2 [==============================] - 0s 9ms/step - loss: 2.9626e-05 - accuracy: 1.0000
Epoch 1119/1500
2/2 [==============================] - 0s 6ms/step - loss: 3.0024e-05 - accuracy: 1.0000
Epoch 1120/1500
2/2 [==============================] - 0s 8ms/step - loss: 2.9484e-05 - accuracy: 1.0000
Epoch 1121/1500
2/2 [==============================] - 0s 8ms/step - loss: 3.0572e-05 - accuracy: 1.0000
Epoch 1122/1500
2/2 [==============================] - 0s 9ms/step - loss: 3.1037e-05 - accuracy: 1.0000
Epoch 1123/1500
2/2 [==============================] - 0s 8ms/step - lo

2/2 [==============================] - 0s 7ms/step - loss: 2.5208e-05 - accuracy: 1.0000
Epoch 1193/1500
2/2 [==============================] - 0s 8ms/step - loss: 2.5598e-05 - accuracy: 1.0000
Epoch 1194/1500
2/2 [==============================] - 0s 13ms/step - loss: 2.3974e-05 - accuracy: 1.0000
Epoch 1195/1500
2/2 [==============================] - 0s 12ms/step - loss: 2.5592e-05 - accuracy: 1.0000
Epoch 1196/1500
2/2 [==============================] - 0s 11ms/step - loss: 2.3479e-05 - accuracy: 1.0000
Epoch 1197/1500
2/2 [==============================] - 0s 12ms/step - loss: 2.4524e-05 - accuracy: 1.0000
Epoch 1198/1500
2/2 [==============================] - 0s 9ms/step - loss: 2.5940e-05 - accuracy: 1.0000
Epoch 1199/1500
2/2 [==============================] - 0s 12ms/step - loss: 2.6349e-05 - accuracy: 1.0000
Epoch 1200/1500
2/2 [==============================] - 0s 8ms/step - loss: 2.6281e-05 - accuracy: 1.0000
Epoch 1201/1500
2/2 [==============================] - 0s 11ms/ste

2/2 [==============================] - 0s 6ms/step - loss: 2.0639e-05 - accuracy: 1.0000
Epoch 1271/1500
2/2 [==============================] - 0s 8ms/step - loss: 2.0560e-05 - accuracy: 1.0000
Epoch 1272/1500
2/2 [==============================] - 0s 8ms/step - loss: 2.1588e-05 - accuracy: 1.0000
Epoch 1273/1500
2/2 [==============================] - 0s 8ms/step - loss: 2.0973e-05 - accuracy: 1.0000
Epoch 1274/1500
2/2 [==============================] - 0s 8ms/step - loss: 1.9668e-05 - accuracy: 1.0000
Epoch 1275/1500
2/2 [==============================] - 0s 11ms/step - loss: 2.0498e-05 - accuracy: 1.0000
Epoch 1276/1500
2/2 [==============================] - 0s 9ms/step - loss: 2.0814e-05 - accuracy: 1.0000
Epoch 1277/1500
2/2 [==============================] - 0s 9ms/step - loss: 2.1338e-05 - accuracy: 1.0000
Epoch 1278/1500
2/2 [==============================] - 0s 9ms/step - loss: 2.0043e-05 - accuracy: 1.0000
Epoch 1279/1500
2/2 [==============================] - 0s 9ms/step - l

2/2 [==============================] - 0s 8ms/step - loss: 1.8218e-05 - accuracy: 1.0000
Epoch 1349/1500
2/2 [==============================] - 0s 9ms/step - loss: 1.8681e-05 - accuracy: 1.0000
Epoch 1350/1500
2/2 [==============================] - 0s 8ms/step - loss: 1.6955e-05 - accuracy: 1.0000
Epoch 1351/1500
2/2 [==============================] - 0s 11ms/step - loss: 1.8767e-05 - accuracy: 1.0000
Epoch 1352/1500
2/2 [==============================] - 0s 8ms/step - loss: 1.8017e-05 - accuracy: 1.0000
Epoch 1353/1500
2/2 [==============================] - 0s 9ms/step - loss: 1.7484e-05 - accuracy: 1.0000
Epoch 1354/1500
2/2 [==============================] - 0s 8ms/step - loss: 1.7395e-05 - accuracy: 1.0000
Epoch 1355/1500
2/2 [==============================] - 0s 7ms/step - loss: 1.6772e-05 - accuracy: 1.0000
Epoch 1356/1500
2/2 [==============================] - 0s 11ms/step - loss: 1.7190e-05 - accuracy: 1.0000
Epoch 1357/1500
2/2 [==============================] - 0s 9ms/step - 

2/2 [==============================] - 0s 9ms/step - loss: 1.5434e-05 - accuracy: 1.0000
Epoch 1427/1500
2/2 [==============================] - 0s 5ms/step - loss: 1.5764e-05 - accuracy: 1.0000
Epoch 1428/1500
2/2 [==============================] - 0s 8ms/step - loss: 1.4166e-05 - accuracy: 1.0000
Epoch 1429/1500
2/2 [==============================] - 0s 8ms/step - loss: 1.4978e-05 - accuracy: 1.0000
Epoch 1430/1500
2/2 [==============================] - 0s 9ms/step - loss: 1.5227e-05 - accuracy: 1.0000
Epoch 1431/1500
2/2 [==============================] - 0s 6ms/step - loss: 1.5102e-05 - accuracy: 1.0000
Epoch 1432/1500
2/2 [==============================] - 0s 7ms/step - loss: 1.5067e-05 - accuracy: 1.0000
Epoch 1433/1500
2/2 [==============================] - 0s 9ms/step - loss: 1.4523e-05 - accuracy: 1.0000
Epoch 1434/1500
2/2 [==============================] - 0s 8ms/step - loss: 1.5136e-05 - accuracy: 1.0000
Epoch 1435/1500
2/2 [==============================] - 0s 10ms/step - l